In [5]:
'''Predicting Mortality across Germany wih different AI Methods
'''

import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import impute
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import dbf
from pygam import GAM, s, f, LinearGAM
import xgboost
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load Data

In [6]:
# Set the working directory
os.chdir("N:/WG_ENRI/20_projects/Noise2NAKO/04_data/grid_prediction/3_output")

In [70]:
# Read training and test data
data_training = pd.read_csv('Prediction/Training_data.csv', sep=',')
data_test = pd.read_csv('Prediction/Test_data.csv', sep=',')

In [71]:
# distinguish between response and input variables and make training and test data 
# remove unnecessary features 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'

output_variable = 'ks_Mortality_17'
output_path = '../5_results/Total Mortality Rate 2017 incl XY/'
coord = True

y = data_training[output_variable]
if coord:
    X = data_training.drop([output_variable, 'ks_Mortality_17', 'id'], axis = 1)   
else:
    X = data_training.drop([output_variable, 'ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)
    
pred_y = data_test[output_variable]
if coord:
    pred_X = data_test.drop([output_variable, 'ks_Mortality_17', 'id'], axis = 1)
else:
    pred_X = data_test.drop([output_variable, 'ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [72]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685 entries, 152 to 265
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   x_sw           685 non-null    int64  
 1   y_sw           685 non-null    float64
 2   x_mp           685 non-null    int64  
 3   y_mp           685 non-null    int64  
 4   ks_foreign_17  685 non-null    float64
 5   ks_inc_hh_17   685 non-null    float64
 6   ks_inc_17      685 non-null    float64
 7   ks_nondipl_17  685 non-null    float64
 8   ks_abi_17      685 non-null    float64
 9   ks_arbl_17     685 non-null    float64
 10  ks_GISD_14     685 non-null    float64
 11  geb_abs        685 non-null    float64
 12  hh_abs         685 non-null    float64
 13  pers_abs       685 non-null    float64
 14  imp_pct05_15   685 non-null    int64  
 15  imp_pct25_15   685 non-null    int64  
 16  imp_pct50_15   685 non-null    float64
 17  imp_mean_15    685 non-null    float64
 18  imp_pct7

# Prediction

In [73]:
def Prediction(method, X_train, X_test, y_train, y_test, pred_X, pred_y, output_variable):
    
    '''Prediction Function'''
    
    if method == 'LR':
        model = linear_model.LinearRegression().fit(X_train, y_train)
    elif method == 'GAM':
        model = GAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9)).fit(X_train, y_train)
    elif method == 'RF':
        model = RandomForestRegressor(n_estimators= 10, max_depth= 10, random_state= 0, max_features='sqrt').fit(X_train, y_train)
    else:
        model = xgboost.XGBRegressor().fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred = model.predict(pred_X)

    result = {'Method': method, 'MSE_train': mean_squared_error(y_train, y_pred_train),
                            'MAE_train': mean_absolute_error(y_train, y_pred_train), 'R_2_train': r2_score(y_train, y_pred_train),
                           'MSE_val': mean_squared_error(y_test, y_pred_test), 'MAE_val': mean_absolute_error(y_test, y_pred_test),
                           'R_2_val': r2_score(y_test, y_pred_test), 'MSE_pred': mean_squared_error(pred_y, y_pred), 'MAE_pred': mean_absolute_error(pred_y, y_pred),
                           'R_2_pred': r2_score(pred_y, y_pred)}
    
    return result, pd.DataFrame(y_pred, columns = [output_variable])

### Prediction with different methods

In [74]:
result= pd.DataFrame(columns=['Method', 'MSE_train', 'MAE_train', 'R_2_train', 'MSE_val', 'MAE_val', 'R_2_val', 'MSE_pred', 'MAE_pred', 'R_2_pred'])

# Linear Regression
ToAppend, y_pred_LR = Prediction(method='LR', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y, output_variable = output_variable)
result = result.append(ToAppend, ignore_index=True)

# GAM
ToAppend, y_pred_GAM = Prediction(method='GAM', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y, output_variable = output_variable)
result = result.append(ToAppend, ignore_index=True)

# Random Forest
ToAppend, y_pred_RF = Prediction(method='RF', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y, output_variable = output_variable)
result = result.append(ToAppend, ignore_index=True)

# XGBoost
ToAppend, y_pred_XGBoost = Prediction(method='XGBoost', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y, output_variable = output_variable)
result = result.append(ToAppend, ignore_index=True)

result.to_csv(output_path + 'PredictionTask_results.csv', index=False, sep=',')

result

,Method,MSE_train,MAE_train,R_2_train,MSE_val,MAE_val,R_2_val,MSE_pred,MAE_pred,R_2_pred
0,LR,0.301840,0.393108,0.916682,0.280661,0.377529,0.922623,1.380903,0.893056,0.592126
1,GAM,0.089763,0.185104,0.975222,0.122811,0.220250,0.966141,1.603605,0.933743,0.526347
2,RF,0.025784,0.087607,0.992883,0.146335,0.210215,0.959656,1.400540,0.896751,0.586326
3,XGBoost,0.000003,0.001227,0.999999,0.048348,0.109419,0.986671,1.126893,0.818418,0.667153


### Merge dataframes to make the final output dataset

In [75]:
def OutputDataframe(data_training, data_test, y_pred, method, output_variable, output_path):
    
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result = data_result.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_pred)],axis=1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + 'prediction_result_'+method+'.csv'), sep=',', index=False)

### Make the difference output

In [76]:
def DiffDataframe(data_training, data_test, y_pred, method, output_variable, output_path):
 
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result[output_variable] = 0
    pred_variable = 'pred'+output_variable
    data_test = pd.concat([data_test, y_pred.rename(columns={output_variable: pred_variable})], axis=1)
    data_test[output_variable] = data_test[output_variable] - data_test[pred_variable]
    data_result = data_result.append(data_test.drop([pred_variable], axis = 1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + 'prediction_result_diff_'+method+'.csv'), sep=',', index=False)

In [77]:
OutputDataframe(data_training,data_test, y_pred_XGBoost, 'XGBoost', output_variable, output_path)
DiffDataframe(data_training,data_test, y_pred_XGBoost, 'XGBoost', output_variable, output_path)

In [78]:
OutputDataframe(data_training,data_test, y_pred_LR, 'LR', output_variable, output_path)
DiffDataframe(data_training,data_test, y_pred_LR, 'LR', output_variable, output_path)

In [79]:
OutputDataframe(data_training,data_test, y_pred_RF, 'RF', output_variable, output_path)
DiffDataframe(data_training,data_test, y_pred_RF, 'RF', output_variable, output_path)

In [80]:
OutputDataframe(data_training,data_test, y_pred_GAM, 'GAM', output_variable, output_path)
DiffDataframe(data_training,data_test, y_pred_GAM, 'GAM', output_variable, output_path)

In [81]:
OutputDataframe(data_training,data_test, pred_y, 'GT', output_variable, output_path)